In [ ]:
import numpy as np
import pandas as pd
import math
import sklearn.preprocessing
import datetime
from TimeBasedCV import TimeBasedCV
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler 
from sklearn.metrics import make_scorer, r2_score

from sklearn.linear_model import HuberRegressor
pd.set_option('display.max_rows', None)
# more

In [8]:
# df = pd.read_csv('factors_2002.csv', parse_dates=['DATE'])
df = pd.read_csv('data/return_predictability_data.csv', parse_dates=['DATE'])
df.head()

,permno,DATE,mvel1,beta,betasq,chmom,dolvol,idiovol,indmom,mom1m,...,name,risk_premium,macro_dp,macro_ep,macro_bm,macro_ntis,macro_tbl,macro_tms,macro_dfy,macro_svar
0,10006,1957-01-31,82249.000,1.122846,1.260784,0.047180,9.569953,0.025742,0.046433,0.044843,...,A C F INDUSTRIES INC,3.3178,3.248434,2.574677,0.567243,0.027992,0.0311,0.0035,0.0072,0.000902
1,10014,1957-01-31,3903.375,0.426734,0.182102,-0.275641,6.237836,0.072103,0.046433,-0.086957,...,ALASKA JUNEAU GOLD MNG CO,6.4038,3.248434,2.574677,0.567243,0.027992,0.0311,0.0035,0.0072,0.000902
2,10022,1957-01-31,9273.250,1.066449,1.137313,-0.025490,7.008844,0.027648,0.046433,-0.060377,...,AMERICAN SAFETY RAZOR CORP,7.0841,3.248434,2.574677,0.567243,0.027992,0.0311,0.0035,0.0072,0.000902
3,10030,1957-01-31,54465.875,0.926038,0.857547,0.018171,9.825337,0.021700,0.046433,0.044633,...,AMERICAN BRAKE SHOE CO,-7.8291,3.248434,2.574677,0.567243,0.027992,0.0311,0.0035,0.0072,0.000902
4,10057,1957-01-31,40250.000,1.247748,1.556875,0.025785,7.901007,0.025506,0.046433,0.086667,...,NATIONAL ACME CO,-12.1262,3.248434,2.574677,0.567243,0.027992,0.0311,0.0035,0.0072,0.000902


In [22]:
#Sort observations by date and stock id
df[df.columns[2:]] = df[df.columns[2:]].astype('float32')
df = df.sort_values(by = ['DATE', 'permno'], ascending = True)
df.head()


,permno,DATE,mvel1,beta,betasq,chmom,dolvol,idiovol,indmom,mom1m,...,macro_dp,macro_ep,macro_bm,macro_ntis,macro_tbl,macro_tms,macro_dfy,macro_svar,Date,macro_mkt-rf
0,10001,2002-01-31,29380.699219,0.024547,0.000603,-0.246853,9.020208,0.029936,-0.032968,6.956505e-03,...,4.274154,3.823618,0.132561,0.011191,0.0165,-0.0027,0.0132,0.002184,200202.0,-1.44
1,10002,2002-01-31,84361.703125,0.149783,0.022435,-0.356072,8.509161,0.058969,0.290509,3.427267e-08,...,4.274154,3.823618,0.132561,0.011191,0.0165,-0.0027,0.0132,0.002184,200202.0,-1.44
2,10012,2002-01-31,65282.878906,2.408649,5.801589,-0.285714,9.949011,0.138741,-0.123827,4.471088e-01,...,4.274154,3.823618,0.132561,0.011191,0.0165,-0.0027,0.0132,0.002184,200202.0,-1.44
3,10019,2002-01-31,10352.500000,1.991428,3.965783,0.214286,7.429473,0.159603,0.039784,4.471088e-01,...,4.274154,3.823618,0.132561,0.011191,0.0165,-0.0027,0.0132,0.002184,200202.0,-1.44
4,10025,2002-01-31,186744.234375,0.138594,0.019208,-0.112840,10.115205,0.070544,0.083402,6.441576e-02,...,4.274154,3.823618,0.132561,0.011191,0.0165,-0.0027,0.0132,0.002184,200202.0,-1.44


In [24]:

df['permno2'] = df['permno'].copy()
df['DATE2'] = df['DATE'].copy()
df = df.set_index(['DATE2','permno2'])

#Make a copy of  the "me" variable (market equity) before rank standartization to use afterwards for value weighting
df['mvel12'] = df['mvel1'].copy()

In [25]:
p=0.3 
df_top = df.groupby('DATE').apply(lambda x: x.nlargest(int(len(x)*p),'mvel1')).reset_index(drop=True)  
df_bottom = df.groupby('DATE').apply(lambda x: x.nsmallest(int(len(x)*p),'mvel1')).reset_index(drop=True)  


C:\Users\drebi\AppData\Local\Temp\ipykernel_39528\2019388754.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_top = df.groupby('DATE').apply(lambda x: x.nlargest(int(len(x)*p),'mvel1')).reset_index(drop=True)
C:\Users\drebi\AppData\Local\Temp\ipykernel_39528\2019388754.py:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_bottom = df.groupby('DATE').apply(lambda x: x.nsmallest(int(len(x)*p),'mvel1'))

In [26]:
#Standardize all independent variables

stdSc = StandardScaler()

features = df.columns[~df.columns.isin(['DATE', 'DATE2', 'mvel1', 'mvel12', 'permno', 'permno2', 'risk_premium'])].tolist()
df[features] = stdSc.fit_transform(df[features].astype(float))


In [5]:
import pickle

with open('return_predictability.pkl', 'rb') as f:
    df = pickle.load(f)
    print(df.head())


ff_research = pd.read_csv('data/ff_factors.csv')
ff_research.head()

df['Date'] = [''.join(x.split('-')[0:2]) for x in df.DATE]

# df.interpolate(method='linear', limit_direction='backward', inplace=True)

df = df.merge(ff_research[['Date', 'macro_mkt-rf']], on = 'Date', how ='inner' )

df['permno2'] = df['permno'].copy()
df['DATE2'] = df['DATE'].copy()
df = df.set_index(['DATE2','permno2'])

#Make a copy of  the "me" variable (market equity) before rank standartization to use afterwards for value weighting
df['mvel12'] = df['mvel1'].copy()


ff3= df[~np.isnan(df['bm'])]
ff3=ff3[~np.isnan(df['mvel1'])]

   permno        DATE      mvel1      beta    betasq     chmom    dolvol  \
0   10006  1957-01-31  82249.000  1.122846  1.260784  0.047180  9.569953   
1   10014  1957-01-31   3903.375  0.426734  0.182102 -0.275641  6.237836   
2   10022  1957-01-31   9273.250  1.066449  1.137313 -0.025490  7.008844   
3   10030  1957-01-31  54465.875  0.926038  0.857547  0.018171  9.825337   
4   10057  1957-01-31  40250.000  1.247748  1.556875  0.025785  7.901007   

    idiovol    indmom     mom1m  ...                        name  \
0  0.025742  0.046433  0.044843  ...        A C F INDUSTRIES INC   
1  0.072103  0.046433 -0.086957  ...   ALASKA JUNEAU GOLD MNG CO   
2  0.027648  0.046433 -0.060377  ...  AMERICAN SAFETY RAZOR CORP   
3  0.021700  0.046433  0.044633  ...      AMERICAN BRAKE SHOE CO   
4  0.025506  0.046433  0.086667  ...            NATIONAL ACME CO   

   risk_premium  macro_dp  macro_ep  macro_bm  macro_ntis  macro_tbl  \
0        3.3178  3.248434  2.574677  0.567243    0.027992     

C:\Users\drebi\AppData\Local\Temp\ipykernel_37640\1677924215.py:26: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  ff3=ff3[~np.isnan(df['mvel1'])]


In [7]:
tscv = TimeBasedCV(train_period=60,
                   val_period=24,
                   test_period=12,
                   freq='months')


X = ff3[['DATE', 'macro_mkt-rf', 'macro_tbl', 'mvel1', 'bm']]
y = ff3[['risk_premium']]

#Empty containers to save results from each window

predictions = []
y_train_list = []
y_val_list = []
y_test_list =[]
dates = []
dic_r2_all = {}

lm = LinearRegression()

for train_index, val_index, test_index in tscv.split(X, first_split_date= datetime.date(2007,1,31), second_split_date= datetime.date(2009,1,31)):

    X_train   = X.loc[train_index].drop('DATE', axis=1)
    y_train = y.loc[train_index]
    lm.fit(X_train, y_train)
    y_pred_train = lm.predict(X_train)
    y_train_list.append(r2_score(y_train, y_pred_train))


    X_val   = X.loc[val_index].drop('DATE', axis=1)
    y_val = y.loc[val_index]
    y_pred_val = lm.predict(X_val)
    y_val_list.append(r2_score(y_val, y_pred_val))


    X_test    = X.loc[test_index].drop('DATE', axis=1)
    y_test  = y.loc[test_index]
    y_pred_test = lm.predict(X_test)
    y_test_list.append(r2_score(y_test, y_pred_test))
    
   
    
    
# predictions_all= np.concatenate(predictions, axis=0)
# y_test_list_all= np.concatenate(y_test_list, axis=0) 
# dates_all= np.concatenate(dates, axis=0)

# R2OOS_LR = 1-sum(pow(y_test_list_all-predictions_all,2))/sum(pow(y_test_list_all,2))
# print("R2OOS Huber Regression: ", R2OOS_LR)


AttributeError: 'str' object has no attribute 'date'

In [30]:
X_train.head()

mvel1        bm  macro_mkt-rf  macro_tbl    mom12m
DATE2      permno2                                                           
2013-01-31 10001    7.610481e+04 -0.107374      1.153359  -0.814497 -0.478548
           10002    4.994548e+04 -0.131432      1.153359  -0.814497  2.362903
           10025    3.276011e+05 -0.234794      1.153359  -0.814497  2.218137
           10026    1.199916e+06 -0.214180      1.153359  -0.814497  0.187221
           10028    6.611568e+04 -0.301330      1.153359  -0.814497 -0.253884

In [ ]:
predictions_all= np.concatenate(predictions, axis=0)
y_test_list_all= np.concatenate(y_test_list, axis=0) 
dates_all= np.concatenate(dates, axis=0)




In [ ]:

predictions_all= np.concatenate(predictions, axis=0)
y_test_list_all= np.concatenate(y_test_list, axis=0) 
dates_all= np.concatenate(dates, axis=0)

#Calculate OOS model performance over the entire test period in line with Gu et al (2020)
R2OOS_LR = 1-sum(pow(y_test_list_all-predictions_all,2))/sum(pow(y_test_list_all,2))
print("R2OOS Huber Regression: ", R2OOS_LR)

In [ ]:

tscv = TimeBasedCV(train_period=60,
                   val_period=24,
                   test_period=12,
                   freq='months')


X = df_top[['DATE','mvel1', 'bm', 'mom12m']]
y = df_top['risk_premium']

#Empty containers to save results from each window
predictions = []
y_test_list =[]
dates = []
dic_r2_all = {}


for train_index, val_index, test_index in tscv.split(X, first_split_date= datetime.date(2008,1,31), second_split_date= datetime.date(2010,1,31)):

    X_train   = X.loc[train_index].drop('DATE', axis=1)
    y_train = y.loc[train_index]
    
    X_val   = X.loc[val_index].drop('DATE', axis=1)
    y_val = y.loc[val_index]

    X_test    = X.loc[test_index].drop('DATE', axis=1)
    y_test  = y.loc[test_index]
    
    #OLS regression with huber loss function
    reg_huber = HuberRegressor(max_iter=1000)
    #No hyperparameters in OLS --> use train and validation set for training
    reg_huber.fit(np.concatenate((X_train, X_val)), np.concatenate((y_train, y_val)))
    #Use test set to generate predictions
    preds = reg_huber.predict(X_test) 
    #Save predictions, dates and the true values of the dependent variable to list  
    predictions.append(preds)
    dates.append(y_test.index)
    y_test_list.append(y_test)
    
    #Calculate OOS model performance the for current window
    r2 = 1-sum(pow(y_test-preds,2))/sum(pow(y_test,2))
    #Save OOS model performance and the respective month to dictionary
    dic_r2_all["r2." + str(y_test.index)] = r2
   
    
    
#Concatenate to get results over the whole OOS test period (Jan 2010-Dec 2019)
predictions_all= np.concatenate(predictions, axis=0)
y_test_list_all= np.concatenate(y_test_list, axis=0) 
dates_all= np.concatenate(dates, axis=0)

#Calculate OOS model performance over the entire test period in line with Gu et al (2020)
R2OOS_LR = 1-sum(pow(y_test_list_all-predictions_all,2))/sum(pow(y_test_list_all,2))
print("R2OOS Huber Regression: ", R2OOS_LR)


In [11]:
df.head()

permno       DATE          mvel1      beta    betasq  \
DATE2      permno2                                                         
2002-01-31 10001     10001 2002-01-31   29380.699219 -1.625643 -0.886911   
           10002     10002 2002-01-31   84361.703125 -1.423729 -0.873616   
           10012     10012 2002-01-31   65282.878906  2.218145  2.645457   
           10019     10019 2002-01-31   10352.500000  1.545477  1.527588   
           10025     10025 2002-01-31  186744.234375 -1.441770 -0.875581   

                       chmom    dolvol   idiovol    indmom     mom1m  ...  \
DATE2      permno2                                                    ...   
2002-01-31 10001   -0.543942 -1.368043 -0.716156 -0.549229 -0.005207  ...   
           10002   -0.783101 -1.555342  0.088333  0.590576 -0.060379  ...   
           10012   -0.629037 -1.027636  2.298814 -0.869382  3.485664  ...   
           10019    0.465824 -1.951048  2.876884 -0.292881  3.485664  ...   
           10025   -0.250490 -0.966725  0.409080 -0.139189  0.450506  ...   

                    risk_premium  macro_dp  macro_ep  macro_bm  macro_ntis  \
DATE2      permno2                                                           
2002-01-31 10001       -3.040000   1.95517  1.793652 -2.925757     0.83144   
           10002       -1.628900   1.95517  1.793652 -2.925757     0.83144   
           10012        8.846900   1.95517  1.793652 -2.925757     0.83144   
           10019      -37.762001   1.95517  1.793652 -2.925757     0.83144   
           10025       12.710700   1.95517  1.793652 -2.925757     0.83144   

                    macro_tbl  macro_tms  macro_dfy  macro_svar         mvel12  
DATE2      permno2                                                              
2002-01-31 10001     0.255957   0.130299   0.628672    -0.14523   29380.699219  
           10002     0.255957   0.130299   0.628672    -0.14523   84361.703125  
           10012     0.255957   0.130299   0.628672    -0.14523   65282.878906  
           10019     0.255957   0.130299   0.628672    -0.14523   10352.500000  
           10025     0.255957   0.130299   0.628672    -0.14523  186744.234375  

[5 rows x 86 columns]